In [1]:
from skyfield.api import load, N, W, wgs84, utc, Angle
from datetime import datetime
from pytz import timezone
est = timezone("US/Eastern")
ts = load.timescale()
de421 = load("de421.bsp")
earth, sun, moon = de421["earth"], de421["sun"], de421["moon"]
# 40°37′55″ N  82°58′37″ W
caledonia_wsg = wgs84.latlon(40.63194 * N, 82.9769 * W)
caledonia = earth + caledonia_wsg

In [2]:
def print_observe(from_obj, obj, tt):
    pos = from_obj.at(tt).observe(obj)
    alt, az, d = pos.apparent().altaz()
    ra, dec, d = pos.radec()
    print(f"{obj.target_name.split()[-1]}\nalt: {alt.degrees:.2f}\t az: {az.degrees:.1f}\n ra: {ra.hours:.3f}h\tdec: {dec.degrees:.1f}")

In [3]:
from skyfield import almanac
a = ts.from_datetime(datetime.strptime("2025-11-11 00:00 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
b = ts.from_datetime(datetime.strptime("2025-11-26 00:00 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
t, y = almanac.find_discrete(a, b, almanac.moon_phases(de421))

for tm, ty, ph in zip(t.astimezone(est),y,[almanac.MOON_PHASES[yi] for yi in y]):
    print(f"{tm.isoformat(' ', 'seconds')} {ph}")
# almanac.moon_phases([a,b])
rt, y = almanac.find_risings(caledonia, sun, a, b)
st, y = almanac.find_settings(caledonia, sun, a, b)
for rtm, stm in zip(rt,st):
    print(f"{rtm.astimezone(est).date()} {rtm.astimezone(est).time().isoformat('seconds')} / {stm.astimezone(est).time().isoformat('seconds')}")

2025-11-12 00:28:08-05:00 Last Quarter
2025-11-20 01:47:16-05:00 New Moon
2025-11-11 07:14:14 / 17:17:16
2025-11-12 07:15:25 / 17:16:21
2025-11-13 07:16:35 / 17:15:28
2025-11-14 07:17:45 / 17:14:36
2025-11-15 07:18:56 / 17:13:47
2025-11-16 07:20:06 / 17:13:00
2025-11-17 07:21:16 / 17:12:14
2025-11-18 07:22:25 / 17:11:31
2025-11-19 07:23:34 / 17:10:50
2025-11-20 07:24:43 / 17:10:10
2025-11-21 07:25:51 / 17:09:33
2025-11-22 07:26:59 / 17:08:57
2025-11-23 07:28:07 / 17:08:24
2025-11-24 07:29:14 / 17:07:53
2025-11-25 07:30:20 / 17:07:24


In [4]:
t = ts.now()
#t = ts.from_datetime(datetime.strptime("2025-08-23 06:06 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
lst = caledonia_wsg.lst_hours_at(t)
print(f"{t.tt_strftime()}\nLST: {Angle(hours=lst)} ({lst:.2f}h)")
print_observe(caledonia, sun, t)
print_observe(caledonia, moon, t)

2025-11-11 16:22:46 TT
LST: 14h 13m 57.75s (14.23h)
SUN
alt: 30.40	 az: 164.9
 ra: 15.115h	dec: -17.5
MOON
alt: 21.17	 az: 277.5
 ra: 9.003h	dec: 19.2
